# **DeepNote**

Here I will try to implement a model that will try to predict the next note in a sequence of note-duration combos.

In [ ]:
!pip list

In [ ]:
#Mounts google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Load in the network input and output

In [ ]:
import numpy as np
with open('/content/gdrive/My Drive/Google Colab/MusicGen/Data/Mozart/net_in_CLBM_L10E_160', 'rb') as filepath:
  network_input = np.load(filepath)
with open('/content/gdrive/My Drive/Google Colab/MusicGen/Data/Mozart/net_out_sounds_CLBM_L10E_160', 'rb') as filepath:
  network_output = np.load(filepath)

In [ ]:
print(network_input.shape)
print(network_output.shape)

(165514, 160, 2)
(165514, 330)


Install and import the needed libraries and modules

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation, Flatten, Bidirectional
from keras.layers import BatchNormalization as BatchNorm
from keras import utils
from keras.callbacks import ModelCheckpoint

model_test = load_model('/content/gdrive/MyDrive/Google Colab/MusicGen/DeepNote_Checkpoints/BM/Ver_1/Round_2/deepNote-25-0.1222.hdf5')
model_test.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 160, 512)          1054720   
_________________________________________________________________
dropout_4 (Dropout)          (None, 160, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 160, 512)          2099200   
_________________________________________________________________
dropout_5 (Dropout)          (None, 160, 512)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 81920)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 330)               27033930  
_________________________________________________________________
activation_2 (Activation)    (None, 330)              

Build the model

In [ ]:
num_classes = len(network_output[0])
#4 0.3 dropout #5 0.2 dropout
model = Sequential()
model.add(Bidirectional(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True)))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(BatchNorm())
model.add(Dropout(0.1))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.1))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.1))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [ ]:
model = load_model('/content/gdrive/MyDrive/Google Colab/MusicGen/DeepNote_Checkpoints/BM/Ver_7/Round_2/deepNote-05-1.6582.hdf5')
model.summary()

In [ ]:
filepath = "/content/gdrive/My Drive/Google Colab/MusicGen/DeepNote_Checkpoints/BM/Ver_7/Round_2/deepNote-{epoch:02d}-{loss:.4f}.hdf5"

BATCH_SIZE = 64
STEPS_PER_EPOCH = len(network_output) / BATCH_SIZE
SAVE_PERIOD = 5
EPOCHS = 150
SAVE_FREQ = int(SAVE_PERIOD * STEPS_PER_EPOCH)

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=False, 
    save_weights_only=False,
    mode='min',
    save_freq=SAVE_FREQ #'epoch'
)
callbacks_list = [checkpoint]

Train the model and plot loss

In [ ]:
from matplotlib import pyplot
#ent/gdrive/My Drive/Google Colab/MusicGen/DeepNote_Checkpoints/BM/Ver_2/deepNote-25-2.9534.hdf5")
model_history = model.fit(network_input, network_output, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)
  
pyplot.plot(model_history.history['loss'], label='train')
pyplot.legend()
pyplot.show()

In [ ]:
model.summary()